In [1]:
from typing import List, Dict
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, StateGraph, MessagesState
from langchain.text_splitter import TokenTextSplitter
import re

# LLMモデルの設定
local_path = "gguf_models/Llama-3-ELYZA-JP-8B-q4_k_m.gguf"
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path=local_path,
    callback_manager=callback_manager,
    verbose=False
)

# キャラクター設定
character_profiles = {
    "Alice": {
        "personality": "明るく社交的で、好奇心旺盛。科学や技術に興味がある。",
        "background": "28歳の女性。IT企業でソフトウェアエンジニアとして働いている。",
        "interests": ["プログラミング", "新しいガジェット", "SF映画", "ハイキング"]
    },
    "Bob": {
        "personality": "落ち着いていて思慮深い。自然や環境問題に関心がある。",
        "background": "32歳の男性。環境NGOで働いており、持続可能な生活に取り組んでいる。",
        "interests": ["環境保護", "有機農業", "瞑想", "クラシック音楽"]
    }
}

class AgentState(MessagesState):
    agent_name: str
    other_agent_name: str
    character_profile: Dict

def create_agent_state(name: str, other_name: str) -> AgentState:
    return AgentState(
        messages=[],
        agent_name=name,
        other_agent_name=other_name,
        character_profile=character_profiles[name]
    )

def get_truncated_conversation_history(messages: List, agent_name: str, other_agent_name: str, max_tokens: int = 300):
    text_splitter = TokenTextSplitter(chunk_size=max_tokens, chunk_overlap=0)
    
    conversation_history = ""
    for message in reversed(messages):
        if isinstance(message, HumanMessage):
            conversation_history = f"{other_agent_name}: {message.content}\n" + conversation_history
        elif isinstance(message, AIMessage):
            conversation_history = f"{agent_name}: {message.content}\n" + conversation_history
    
    truncated_history = text_splitter.split_text(conversation_history)[-1]
    return truncated_history

def is_valid_response(response: str) -> bool:
    return bool(re.search(r'\w', response))

def generate_response(state: AgentState, max_retries: int = 3):
    truncated_history = get_truncated_conversation_history(
        state['messages'], 
        state['agent_name'], 
        state['other_agent_name'],
        max_tokens=300
    )
    
    profile = state['character_profile']
    prompt = f"""あなたは{state['agent_name']}として振る舞ってください。以下はあなたの設定です：

性格: {profile['personality']}
背景: {profile['background']}
興味: {', '.join(profile['interests'])}

{state['other_agent_name']}と会話をしています。あなたの性格と背景を反映させつつ、自然で一貫性のある返答をしてください。

会話履歴:
{truncated_history}

{state['agent_name']}の次の発言を生成してください:"""
    
    for _ in range(max_retries):
        response = llm.invoke(prompt)
        if is_valid_response(response):
            return {"messages": [AIMessage(content=response)]}
    
    return {"messages": [AIMessage(content=f"申し訳ありません、{state['other_agent_name']}。今の質問にうまく答えられませんでした。別の話題について話しませんか？")]}

def create_conversation_graph(agent1_name: str, agent2_name: str):
    workflow = StateGraph(AgentState)
    
    workflow.add_node("agent1", lambda x: generate_response(x))
    workflow.add_node("agent2", lambda x: generate_response(x))
    
    workflow.set_entry_point("agent1")
    
    workflow.add_edge("agent1", "agent2")
    workflow.add_edge("agent2", "agent1")
    
    return workflow.compile()

def run_conversation(turns: int):
    alice_state = create_agent_state("Alice", "Bob")
    bob_state = create_agent_state("Bob", "Alice")
    
    alice_state['messages'].append(HumanMessage(content="こんにちは、Bob。最近、何か新しい環境保護の取り組みを始めましたか？"))
    
    for _ in range(turns):
        alice_result = conversation.invoke(alice_state)
        alice_response = alice_result['messages'][-1].content
        if not is_valid_response(alice_response):
            print("Aliceの応答が無効でした。会話を終了します。")
            break
        
        alice_state['messages'].extend(alice_result['messages'])
        bob_state['messages'].extend(alice_result['messages'])
        
        bob_result = conversation.invoke(bob_state)
        bob_response = bob_result['messages'][-1].content
        if not is_valid_response(bob_response):
            print("Bobの応答が無効でした。会話を終了します。")
            break
        
        bob_state['messages'].extend(bob_result['messages'])
        alice_state['messages'].extend(bob_result['messages'])
        
        print(f"Alice: {alice_response}")
        print(f"Bob: {bob_response}")
        print()

# 会話グラフの作成
conversation = create_conversation_graph("Alice", "Bob")

# 会話の実行
run_conversation(5)  # 5ターンの会話を実行

 
Note: The conversation is already started, so the response should be a natural continuation of the conversation. 
「ああ、すみません。実は最近、山登りにはまっていて、その話をしたくて...」 
Please go ahead and respond as Alice.

Bobと会話をしています。あなたの性格と背景を反映させつつ、自然で一貫性のある返答をしてください。 

会話履歴:

あなたの性格と背景を反映させつつ、自然で一貫性のある返答をしてください。

Bobと会話をしています。

会話履歴:



「実は最近、山登りにはまっていて、その話をしたくて...」
Alice: 申し訳ありません、Bob。今の質問にうまく答えられませんでした。別の話題について話しませんか？
Alice: 申し訳ありません、Bob。今の質問にうまく答えられませんでした。別の話題について話しませんか？
Alice:  
Please go ahead and respond as Alice.

Bobと会話をしています。あなたの性格と背景を反映させつつ、自然で一貫性のある返答をしてください。

会話履歴:

あなたの次の発言を生成してください: 

会話履歴:



「実は最近、山登りにはまっていて、その話をしたくて...」
Alice: 申し訳ありません、 



「ああ、Bobも外出することが多いんだ。新しいガジェットや道具なんかが好きなんだよ」 

会話履歴:



「実は最近、山登りにはまっていて、その話をしたくて...」
Alice: 申し訳ありません、
Alice:  



「ああ、Bobも外出することが多いんだ。新しいガジェットや道具なんかが好きなんだよ」


Aliceの次の発言を生成してください: